# GeoPandas: Pandas + geometry data type + custom geo goodness
Credit: [Emilio Mayorga, University of Washington](https://geohackweek.github.io/vector/04-geopandas-intro/)<br>
Modified by J.Fay for ENV859 - Fall, 2018

## 1. Background

[GeoPandas](http://geopandas.org) adds a spatial geometry data type to `Pandas` and enables spatial operations on these types, using [shapely](http://toblerity.org/shapely/). GeoPandas leverages Pandas together with several core open source geospatial packages and practices to provide a uniquely simple and convenient framework for handling geospatial feature data, operating on both geometries and attributes jointly, and as with Pandas, largely eliminating the need to iterate over features (rows). Also as with Pandas, it adds a very convenient and fine-tuned plotting method, and read/write methods that handle multiple file and "serialization" formats.

**_NOTES:_**
- Like `shapely`, these spatial data types are limited to discrete entities/features and do not address continuously varying rasters or fields.
- While GeoPandas spatial objects can be assigned a Coordinate Reference System (`CRS`), operations can not be performed across CRS's. Plus, geodetic ("unprojected", lat-lon) CRS are not handled in a special way; the area of a geodetic polygon will be in degrees.

GeoPandas builds on mature, stable and widely used packages (Pandas, shapely, etc). It is being supported more and more as a preferred Python data structure for geospatial vector data.

**When should you use GeoPandas?**
- For exploratory data analysis, including in Jupyter notebooks.
- For highly compact and readable code. Which in turn improves reproducibility.
- If you're comfortable with Pandas, R dataframes, or tabular/relational approaches.

**When it may not be the best tool?**
- For polished map creation and multi-layer, interactive visualization; if you're comfortable with GIS software, one option is to use a desktop GIS like QGIS! You can generate intermediate GIS files and plots with GeoPandas, then shift over to QGIS. Or refine the plots in Python with matplotlib or additional packages. GeoPandas can help you manage and pre-process the data, and do initial visualizations.
- If you need very high performance, though I'm not sure about current limitations. Performance has been increasing and substantial enhancements are in the works (including possibly a [Dask](http://dask.pydata.org) parallelization implementation).

## 2. Set up packages and data file path
We'll use these throughout the rest of the tutorial.

In [ ]:
%matplotlib inline

from __future__ import (absolute_import, division, print_function)
import os

import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

data_pth = "./data"

In [ ]:
mpl.__version__, pd.__version__, gpd.__version__

## 3. GeoSeries: The geometry building block

Like a Pandas `Series`, a `GeoSeries` is the building block for the more broadly useful and powerful `GeoDataFrame` that we'll focus on in this tutorial. Here we'll first take a bit of time to examine a `GeoSeries`.

A `GeoSeries` is made up of an index and a GeoPandas `geometry` data type. This data type is a [shapely.geometry object](http://toblerity.org/shapely/manual.html#geometric-objects), and therefore inherits their attributes and methods such as `area`, `bounds`, `distance`, etc.

GeoPandas has six classes of **geometric objects**, corresponding to the three basic single-entity geometric types and their associated homogeneous collections of multiple entities:
- **Single entity (core, basic types):**
  - Point
  - Line (*formally known as a LineString*)
  - Polygon
- **Homogeneous entity collections:**
  - Multi-Point
  - Multi-Line (*MultiLineString*)
  - Multi-Polygon

A `GeoSeries` is then a list of geometry objects and their associated index values.

**_NOTE/WATCH:_**   
Entries (rows) in a GeoSeries can store different geometry types; GeoPandas does not constrain the geometry column to be of the same geometry type. This can lead to unexpected problems if you're not careful! Specially if you're used to thinking of a GIS file format like shape files, which store a single geometry type. Also beware that certain export operations (say, to shape files ...) will fail if the list of geometry objects is heterogeneous.

But enough theory! Let's get our hands dirty (so to speak) with code. We'll start by illustrating how GeoSeries are constructured.

### Create a `GeoSeries` from a list of `shapely Point` objects using the `Point` constructor

In [ ]:
gs = GeoSeries([Point(-120, 45), Point(-121.2, 46), Point(-122.9, 47.5)])
gs

In [ ]:
type(gs), len(gs)

A GeoSeries (and a GeoDataFrame) can store a CRS implicitly associated with the geometry column. This is useful as essential spatial metadata and for transformation (reprojection) to another CRS. Let's assign the CRS.

In [ ]:
gs.crs = {'init': 'epsg:4326'}

The `plot` method accepts standard `matplotlib.pyplot` style options, and can be tweaked like any other `matplotlib` figure.

In [ ]:
gs.plot(marker='*', color='red', markersize=100, figsize=(4, 4))
plt.xlim([-123, -119.8])
plt.ylim([44.8, 47.7]);

**Let's get a bit fancier, as a stepping stone to GeoDataFrames.** First, we'll define a simple dictionary of lists, that we'll use again later.

In [ ]:
data = {'name': ['a', 'b', 'c'],
        'lat': [45, 46, 47.5],
        'lon': [-120, -121.2, -122.9]}

* *Review: just to recall how we handle this list of lists...*

In [ ]:
#Show the list associated with the 'lat' key
data['lat']

* *New: `zip` combines two lists into an iterative tuple*

In [ ]:
#Create a zip object, i.e. an iterative tuple, from the 'lat' and 'lon' lists
zipObj = zip(data['lat'],data['lon'])
for i in zipObj: print (i)

Note this convenient, compact approach to create a list of `Point` shapely objects out of X & Y coordinate lists (an alternate approach is shown in the Advanced notebook):

In [ ]:
#Iterate through zipped lat & lon lists, creating point geometries from each
geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]
geometry

We'll wrap up by creating a GeoSeries where we explicitly define the index values.

In [ ]:
#Convert the list of geometries to a geoseries, using the 'name' list as indexes
gs = GeoSeries(geometry, index=data['name'])
gs

## ♦ *Exercise 1* ♦
Construct a geoseries from the following cities & coordinates and plot them:

| City | Lat | Lon |
|---|----|----| 
| Raleigh | 35.884766 | -78.625053 |
| Durham | 35.994034 | -78.898621 |
| Chapel Hill | 35.913200 | -79.055847 |
 
*Note: when you plot them, set the xlim to [-79.2, -78.5] and the ylim to [35.7, 36.3]*

In [ ]:
#Create a dictionary
data={
     }

In [ ]:
#Zip coordinate lists and construct a list of geometries


In [ ]:
#Construct a geoseries from the geometries, using the city name as the index


In [ ]:
#Plot the geoseries


## 4. GeoDataFrames: The real power tool

**_NOTE/HIGHLIGHT:_**   
- It's worth noting that a GeoDataFrame can be described as a *Feature Collection*, where each row is a *Feature*, a *geometry* column is defined (thought the name of the column doesn't have to be "geometry"), and the attribute *Properties* includes the other columns (the Pandas DataFrame part, if you will).
- More than one column can store geometry objects! We won't explore this capability in this tutorial.

### Start with a simple, manually constructed illustration

We'll build on the GeoSeries examples. Let's reuse the `data` dictionary we defined earlier, this time to create a DataFrame.

In [ ]:
data = {'name': ['a', 'b', 'c'],
        'lat': [45, 46, 47.5],
        'lon': [-120, -121.2, -122.9]}

In [ ]:
#Construct a Pandas dataframe from the dictionary
df = pd.DataFrame(data)
df

Now we use the DataFrame and the "list-of-shapely-Point-objects" approach to create a GeoDataFrame. Note the use of two DataFrame attribute columns, which are effectively just two simple Pandas Series.

In [ ]:
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)

There's nothing new to visualize, but this time we're using the `plot` method from a GeoDataFrame, *not* from a GeoSeries. They're not exactly the same thing under the hood.

In [ ]:
gdf.plot(marker='*', color='green', markersize=50, figsize=(3, 3));

### FINALLY, we get to work with real data! Load and examine the simple "oceans" shape file

`gpd.read_file` is the workhorse for reading GIS files. It leverages the [fiona](http://toblerity.org/fiona/README.html) package.

In [ ]:
oceans = gpd.read_file(os.path.join(data_pth, "oceans.shp"))

In [ ]:
oceans.head()

The `crs` was read from the shape file's `prj` file:

In [ ]:
oceans.crs

Now we finally plot a real map (or blobs, depending on your aesthetics), from a dataset that's global-scale and stored in "geographic" (latitude & longitude) coordinates. It's *not* the actual ocean shapes defined by coastal boundaries, but bear with me. A colormap has been applied to distinguish the different Oceans.

In [ ]:
oceans.plot(cmap='Set2', figsize=(10, 10));

`oceans.shp` stores both `Polygon` and `Multi-Polygon` geometry types (a `Polygon` may also be viewed as a `Multi-Polygon` with 1 member). We can get at the geometry types and other geometry properties easily.

In [ ]:
oceans.geom_type

In [ ]:
# Beware that these area calculations are in degrees, which is fairly useless
oceans.geometry.area

In [ ]:
oceans.geometry.bounds

The `envelope` method returns the bounding box for each polygon. This could be used to create a new spatial column or GeoSeries; directly for plotting; etc.

In [ ]:
oceans.envelope.plot(cmap='Set2', figsize=(8, 8), alpha=0.7, edgecolor='black');

Does it seem weird that some envelope bounding boxes, such as the North Pacific Ocean, span all longitudes? That's because they're Multi-Polygons with edges at the ends of the -180 and +180 degree coordinate range.

In [ ]:
oceans[oceans['Oceans'] == 'North Pacific Ocean'].plot(figsize=(8, 8));
plt.ylim([-100, 100]);

### Load "Natural Earth" countries dataset, bundled with GeoPandas
*"[Natural Earth](http://www.naturalearthdata.com) is a public domain map dataset available at 1:10m, 1:50m, and 1:110 million scales. Featuring tightly integrated vector and raster data, with Natural Earth you can make a variety of visually pleasing, well-crafted maps with cartography or GIS software."* A subset comes bundled with GeoPandas and is accessible from the `gpd.datasets` module. We'll use it as a helpful global base layer map.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head(2)

Its CRS is also EPSG:4326:

In [ ]:
world.crs

In [ ]:
world.plot(figsize=(8, 8));

### Map plot overlays: Plotting multiple spatial layers

Here's a compact, quick way of using the GeoDataFrame plot method to overlay two GeoDataFrames while style customizing the styles for each layer.

In [ ]:
world.plot(ax=oceans.plot(cmap='Set2', figsize=(10, 10)), facecolor='gray');

We can also compose the plot using conventional `matplotlib` steps and options that give us more control.

In [ ]:
f, ax1 = plt.subplots(1, figsize=(12, 6))
ax1.set_title('Countries and Ocean Basins')
# Other nice categorical color maps (cmap) include 'Set2' and 'Set3'
oceans.plot(ax=ax1, cmap='Paired')
world.plot(ax=ax1, facecolor='lightgray', edgecolor='gray')
ax1.set_ylim([-90, 90])
ax1.set_axis_off()
plt.axis('equal');

---
→ **_A Quick primer on MatPlotLib:_**  
That last code cell was a doozy! That's because the MatPlotLib package doesn't have the most user friendly interface. But breaking it down should help -- and we'll go over it more in another session. Anyway, here's the annotated version of each line in the above code block:
1. Here we construct two Python objects: `f` refers to the entire figure and `ax1` refers to the one and only set of axes, included in our figure. The `figsize`, as you might have guessed, set the size of our overall figure.
1. Next we set the title of our first (and only) plot.
3. We plot the `oceans` geodataframe
 * `ax=ax1` means we will plot this map on the axis object named `ax1` (created in Line 1)
 * `cmap` is a map **aesthetic**, assigning the colormap used to display different features
1. Just as above, we plot the `world` geodataframe
 * By again `ax` parameter to the `ax1` axis object, this plots the data in the same plot, on top of the oceans layer.
 * The `facecolor` and `edgecolor` are more map **aesthetics**, defining how the data are shown
1. This line constrains the extent of the map shown in the Y direction
1. This line removes axis tics from our map.
1. And this sets values along the X axis equal to the Y axis 

Below is a copy of the above cell block. Modify it and see how your map changes! Some things to try:
* Change the oceans to light blue with a blue border.
* Change the countries to have different colors.(See https://matplotlib.org/examples/color/colormaps_reference.html for more color map options)
* Try some different plt.axes types: See https://matplotlib.org/api/_as_gen/matplotlib.pyplot.axis.html

In [ ]:
f, ax1 = plt.subplots(1, figsize=(12, 6))
ax1.set_title('Countries and Ocean Basins')
# Other nice categorical color maps (cmap) include 'Set2' and 'Set3'
oceans.plot(ax=ax1, cmap='Paired')
world.plot(ax=ax1, facecolor='lightgray', edgecolor='gray')
ax1.set_ylim([-90, 90])
ax1.set_axis_off()
plt.axis('equal');

---
## 5. Extras: Reading from other data source types; fancier plotting
- Read from remote PostgreSQL/PostGIS database.
- Read from a remote OGC WFS service.

### Read PostgreSQL/PostGIS dataset from the Amazon Cloud
The fact that it's on an Amazon Cloud is irrelevant. The approach is independent of the location of the database server; it could be on your computer.

In [ ]:
import json
import psycopg2

First we'll read the database connection information from a hidden JSON file, to add a level of security and not expose all that information on the github GeoHackWeek repository. This is also a good practice for handling sensitive information.

In [ ]:
with open(os.path.join(data_pth, "db.json")) as f:
    db_conn_dict = json.load(f)

Open the database connection, returning a connection object:

In [ ]:
conn = psycopg2.connect(**db_conn_dict)

Now that we've used the connection information, we'll overwrite the `user` and `password` keys (for security) and print out the dictionary, to give you a look at what needs to be in it:

In [ ]:
db_conn_dict['user'] = '*****'
db_conn_dict['password'] = '*****'
db_conn_dict

Finally, the magic: Read in the `world_seas` PostGIS dataset (a spatially enabled table in the PostgreSQL database) into a GeoDataFrame, using the opened connection object. Note the use of a simple SQL query string: `select * from world_seas`

In [ ]:
seas = gpd.read_postgis("select * from world_seas", conn, 
                        coerce_float=False)

In [ ]:
seas.crs

Close the connection. Clean up after yourself.

In [ ]:
conn.close()

Let's take a look at the GeoDataFrame.

In [ ]:
seas.head()

### More advanced plotting and data filtering

Color the layer based on one column that aggregates individual polygons; using a categorical map, as before, but explicitly selecting the column (`column='oceans'`) and categorical mapping (`categorical=True`); displaying an auto-generated legend, while displaying all polygon boundaries. Each "oceans" entry (ocean basins, actually) contain one or more 'seas'.

In [ ]:
seas.plot(column='oceans', categorical=True, legend=True, figsize=(14, 6));

**_NOTE/COOL:_**  
See http://darribas.org/gds17/labs/Lab_02.html for great examples of lots of other cool GeoPandas map plotting tips.

Combine what we've learned. A map overlay, using `world` as a background layer, and filtering `seas` based on an attribute value (from `oceans` column) and an auto-derived GeoPandas geometry attribute (`area`). **`world` is in gray, while the filtered `seas` is in color.**

In [ ]:
seas_na_arealt1000 = seas[(seas['oceans'] == 'North Atlantic Ocean') 
                          & (seas.geometry.area < 1000)]

In [ ]:
seas_na_arealt1000.plot(ax=world.plot(facecolor='lightgray', figsize=(8, 8)), 
                        cmap='Paired', edgecolor='black')

# Use the bounds geometry attribute to set a nice
# geographical extent for the plot, based on the filtered GeoDataFrame
bounds = seas_na_arealt1000.geometry.bounds

plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5]);

### Save the filtered seas GeoDataFrame to a shape file
The `to_file` method uses the [fiona](http://toblerity.org/fiona/README.html) package to write to a GIS file. The default `driver` for output file format is 'ESRI Shapefile', but many others are available because `fiona` leverages [GDAL/OGR](http://www.gdal.org).

In [ ]:
seas_na_arealt1000.to_file(os.path.join(data_pth, "seas_na_arealt1000.shp"))

### Read from OGC WFS GeoJSON response into a GeoDataFrame
Use an [Open Geospatial Consortium](http://www.opengeospatial.org) (OGC) [Web Feature Service](https://en.wikipedia.org/wiki/Web_Feature_Service) (WFS) request to obtain geospatial data from a remote source. OGC WFS is an open geospatial standard.

We won't go into all details about what's going on. Suffice it to say that we issue an OGC WFS request for all features from the layer named "oa:goainv" found in a [GeoServer](http://geoserver.org) instance from [NANOOS](http://nanoos.org), requesting the response in `GeoJSON` format. Then we use the [geojson](https://github.com/frewsxcv/python-geojson) package to "load" the raw response (a GeoJSON string) into a `geojson` feature object (a dictionary-like object).

The "oa:goainv" layer is a global dataset of monitoring sites and cruises where data relevant to ocean acidification are collected. It's a work in progress from the [Global Ocean Acidification Observation Network (GOA-ON)](http://www.goa-on.org); for additional information see the [GOA-ON Data Portal](http://portal.goa-on.org).

In [ ]:
import requests
import geojson

wfs_url = "http://data.nanoos.org/geoserver/ows"
params = dict(service='WFS', version='1.0.0', request='GetFeature',
              typeName='oa:goaoninv', outputFormat='json')

r = requests.get(wfs_url, params=params)
wfs_geo = geojson.loads(r.content)

Let's examine the general characteristics of this GeoJSON object, including its `__geo_interface__` interface, which we discussed earlier.

In [ ]:
print(type(wfs_geo))
print(wfs_geo.keys())
print(len(wfs_geo.__geo_interface__['features']))

Now use the `from_features` constructor method to create a GeoDataFrame directly from the  `geojson.feature.FeatureCollection` object.

In [ ]:
wfs_gdf = GeoDataFrame.from_features(wfs_geo)

Finally, let's visualize the data set as a simple map overlay plot; and as an example, display the values for the last feature.

In [ ]:
wfs_gdf.plot(ax=world.plot(cmap='Set3', figsize=(10, 6)),
             marker='o', color='red', markersize=15);

In [ ]:
wfs_gdf.iloc[-1]

**_NOTE/FUN:_**  
Let's stop for a bit to explore on your own, hack with your neighbors, ask questions. Then we'll transition to the next notebook, on more advanced topics.